# Creating an Autogewn Agent that talk to Notion MCP



In [7]:
from autogen_ext.tools.mcp import mcp_server_tools, SseServerParams
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent

params = SseServerParams(
    url="https://mcp.notion.com/sse",        # SSE endpoint
    timeout=10.0,                            # HTTP connect timeout
    sse_read_timeout=300.0                   # SSE read timeout
)
tools = await mcp_server_tools(params)  

  + Exception Group Traceback (most recent call last):
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_52955/1210251342.py", line 10, in <module>
  |     tools = await mcp_server_tools(params)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_factory.py", line 193, in mcp_server_tools
  |     async with create_mcp_server_session(server_params) as temp_session:
  |   File "/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/contextlib.py", line 199, in __aenter__
  |     return await anext(self.gen)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_session.py", line 27, in create_mcp_server_session
  |     async with sse_clien

In [6]:
import asyncio
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

async def main() -> None:
    server_params = StdioServerParams(
        command="npx", args=["-y", "@notionhq/notion-mcp-server"]
    )

    # Get all available tools from the server
    tools = await mcp_server_tools(server_params)

    for tool in tools:
        print(f"{tool.name} - {tool.description}")


await main()

/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


FormatNotSupportedError: Unknown format `json` for `icon` in `GeneratedModel`

In [2]:
from mcp.client.auth import OAuthClientProvider, TokenStorage


In [13]:
"""
notion_oauth_client.py
Run once → browser pops → grant access → paste callback → tokens cached.
Subsequent runs reuse the refresh token so no more browser pop-ups.
"""

import asyncio, json, os, webbrowser, aiohttp
from urllib.parse import urlparse, parse_qs
from pydantic import AnyUrl

from mcp.client.auth import OAuthClientProvider, TokenStorage
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
from autogen_ext.tools.mcp import mcp_server_tools, StreamableHttpServerParams
from mcp.client.auth import (
    OAuthClientProvider,
    TokenStorage
)
from mcp.shared.auth import OAuthClientMetadata
from pydantic import AnyUrl


# ----  Tiny file-based token cache  -----------------------------------------
class FileTokenStore(TokenStorage):
    def __init__(self, path=".notion_oauth_tokens.json"):
        self.path = path
    async def get_tokens(self):         # type: ignore
        if os.path.exists(self.path):
            return json.load(open(self.path))
    async def set_tokens(self, tokens): # type: ignore
        json.dump(tokens, open(self.path, "w"))
    async def get_client_info(self):    # type: ignore
        return None                     # dynamic registration each time
    async def set_client_info(self, info): pass

# ----  Handlers -------------------------------------------------------------
async def handle_redirect(auth_url: str) -> None:
    print(f"Opening browser → {auth_url}")
    webbrowser.open(auth_url)

async def handle_callback() -> tuple[str, str | None]:
    print("Paste the full callback URL you were redirected to:")
    callback_url = input().strip()
    qs = parse_qs(urlparse(callback_url).query)
    return qs["code"][0], qs.get("state", [None])[0]

# ----  Build the auth provider  --------------------------------------------
oauth = OAuthClientProvider(
    server_url="https://mcp.notion.com",
    client_metadata=OAuthClientMetadata(
        client_name="AdmonishNotionMCPClient",
        redirect_uris=[AnyUrl("http://localhost:3000/callback")],
        grant_types=["authorization_code", "refresh_token"],
        response_types=["code"],
        scope="read write"
    ),
    storage=FileTokenStore(),
    redirect_handler=handle_redirect,    # your existing function
    callback_handler=handle_callback     # your existing function
)

# ----  Use it with Streamable HTTP  ----------------------------------------
async def main():
    params = StreamableHttpServerParams(
        url="https://mcp.notion.com/mcp",
        auth_provider=oauth,
        timeout=15.0,
    )
    tools = await mcp_server_tools(params)
    print("✅  Connected! first five tools:")
    for t in tools[:5]:
        print("   -", t.name)

await main()


  + Exception Group Traceback (most recent call last):
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_82176/2683858048.py", line 74, in <module>
  |     await main()
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_82176/2683858048.py", line 69, in main
  |     tools = await mcp_server_tools(params)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_factory.py", line 198, in mcp_server_tools
  |     async with create_mcp_server_session(server_params) as temp_session:
  |   File "/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/contextlib.py", line 217, in __aexit__
  |     await self.gen.athrow(typ, value, traceback)
  |   File "/Users/hugoevers/VScode-projects/ad

In [1]:
from autogen_ext.tools.mcp import mcp_server_tools, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from dotenv import load_dotenv, find_dotenv
import os

# Load environment variables from .env file
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



params = StreamableHttpServerParams(
    url="https://mcp.notion.com/mcp",
    headers={                               # <- KEY PART
    "Authorization": f"Bearer {os.environ['NOTION_TOKEN']}",
    "Notion-Version": "2022-06-28"      # same header Notion’s README shows
},
    timeout=10.0,
    sse_read_timeout=300.0

)

tools = await mcp_server_tools(params)

# for i, tool in enumerate(tools[:10], 1):
#     tool_name = getattr(tool, 'name', str(tool))
#     tool_desc = getattr(tool, 'description', 'No description')
#     print(f"   {i}. {tool_name}: {tool_desc[:50]}...")


# prompt = "You are an agent that can interact with Notion MCP. You can use the tools provided to perform various tasks related to Notion. Your responses should be concise and relevant to the task at hand. If you need to use a tool, make sure to provide a clear explanation of what you are doing and why."

# agent = AssistantAgent(
#     name="notion-agent",
#     model_client=OpenAIChatCompletionClient(
#         model="gpt-4o-mini", 
#         api_key=OPENAI_API_KEY
#     ),
#     system_message=prompt,
#     tools=tools
# )

  + Exception Group Traceback (most recent call last):
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_82176/2107984663.py", line 26, in <module>
  |     tools = await mcp_server_tools(params)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_factory.py", line 198, in mcp_server_tools
  |     async with create_mcp_server_session(server_params) as temp_session:
  |   File "/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/contextlib.py", line 217, in __aexit__
  |     await self.gen.athrow(typ, value, traceback)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_session.py", line 43, in create_mcp_server_session
  |     async

In [8]:
import asyncio
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

async def main() -> None:
    server_params = StdioServerParams(
        command="npx", args=["-y", "@notionhq/notion-mcp-server"]
    )

    # Get all available tools from the server
    tools = await mcp_server_tools(server_params)

    for tool in tools:
        print(f"{tool.name} - {tool.description}")


await main()

API-get-user - Retrieve a user
Error Responses:
400: 400
API-get-users - List all users
Error Responses:
400: 400
API-get-self - Retrieve your token's bot user
API-post-database-query - Query a database
API-post-search - Search by title
API-get-block-children - Retrieve block children
API-patch-block-children - Append block children
API-retrieve-a-block - Retrieve a block
API-update-a-block - Update a block
API-delete-a-block - Delete a block
API-retrieve-a-page - Retrieve a page
API-patch-page - Update page properties
API-post-page - Create a page
API-create-a-database - Create a database
API-update-a-database - Update a database
API-retrieve-a-database - Retrieve a database
API-retrieve-a-page-property - Retrieve a page property item
API-retrieve-a-comment - Retrieve comments
API-create-a-comment - Create comment


In [9]:
import asyncio, json, os, webbrowser, secrets
from urllib.parse import urlparse, parse_qs
from pydantic import AnyUrl
from mcp.client.auth import OAuthClientProvider, TokenStorage
from mcp.shared.auth import OAuthClientMetadata            # <-- right import
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
from autogen_ext.tools.mcp import (
    StreamableHttpServerParams,              # only for metadata later
    mcp_server_tools, StreamableHttpMcpToolAdapter
)

In [ ]:
"""
WORKING NOTION MCP + AUTOGEN INTEGRATION
=====================================
This cell contains all the working code for:
1. Connecting to Notion MCP via stdio (reliable method)
2. Creating an AutoGen agent with Notion tools
3. Testing the integration

Run this cell to get a fully functional Notion-enabled AutoGen agent.
"""

import asyncio
import os
from dotenv import load_dotenv, find_dotenv

# AutoGen imports
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Load environment variables
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

async def connect_notion_mcp():
    """
    Connect to Notion MCP server using the working stdio approach.
    Returns tools that can be used with AutoGen agents.
    """
    try:
        # Use the stdio approach with npx - this works reliably
        server_params = StdioServerParams(
            command="npx", 
            args=["-y", "@notionhq/notion-mcp-server"]
        )

        print("🔄 Connecting to Notion MCP via stdio...")
        tools = await mcp_server_tools(server_params)

        print(f"✅ Connected successfully!")
        print(f"📋 Available tools: {len(tools)}")
        
        # Show first few tools
        print("\n🛠️  Sample tools:")
        for i, tool in enumerate(tools[:5], 1):
            print(f"  {i}. {tool.name}")
            
        return tools
        
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        raise

async def create_notion_agent(tools):
    """Create AutoGen agent with Notion MCP tools"""
    
    if not OPENAI_API_KEY:
        raise ValueError("❌ OPENAI_API_KEY not found in environment")
    
    print("✅ OpenAI API key loaded")

    # Create AutoGen agent with Notion MCP tools
    agent = AssistantAgent(
        name="notion_assistant",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o-mini", 
            api_key=OPENAI_API_KEY
        ),
        system_message="""You are a helpful assistant that can interact with Notion using MCP tools. 

You can:
- Search for pages and databases
- Query databases 
- Retrieve user information
- Get page/block details
- Create and update content

Always be helpful and explain what you're doing when using tools.
""",
        tools=tools
    )

    print(f"🤖 Created AutoGen agent '{agent.name}' with {len(tools)} Notion tools")
    return agent

async def test_notion_agent(agent):
    """Test the agent with a simple request"""
    try:
        print("🧪 Testing the notion_agent...")
        
        # Create a simple test message
        message = TextMessage(
            content="Hello! Can you tell me what tools you have available for working with Notion?",
            source="user"
        )
        
        # Get response from agent
        response = await agent.on_messages(
            messages=[message], 
            cancellation_token=CancellationToken()
        )
        
        print("🤖 Agent response:")
        print(response.inner)
        return response
        
    except Exception as e:
        print(f"❌ Test failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# MAIN EXECUTION
async def main():
    """Main function to set up everything"""
    print("=" * 60)
    print("🚀 NOTION MCP + AUTOGEN SETUP")
    print("=" * 60)
    
    try:
        # Step 1: Connect to Notion MCP and get tools
        notion_tools = await connect_notion_mcp()
        
        # Step 2: Create AutoGen agent with Notion tools
        notion_agent = await create_notion_agent(notion_tools)
        
        # Step 3: Test the agent
        print("\n" + "=" * 50)
        test_response = await test_notion_agent(notion_agent)
        print("=" * 50)
        
        if test_response:
            print("✅ Full setup completed successfully!")
            print(f"📝 Ready to interact with Notion via '{notion_agent.name}'!")
            return notion_agent, notion_tools
        else:
            print("❌ Setup failed during testing")
            return None, None
            
    except Exception as e:
        print(f"❌ Setup failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Run the complete setup
notion_agent, notion_tools = await main()

if notion_agent:
    print(f"\n🎉 SUCCESS! Agent '{notion_agent.name}' is ready to use.")
    print(f"📊 Available tools: {len(notion_tools)}")
else:
    print("\n💥 Setup failed. Check the errors above.")

🔄 Connecting to Notion MCP via stdio...
✅ Connected successfully!
📋 Available tools: 19

🛠️  Sample tools:
  1. API-get-user
  2. API-get-users
  3. API-get-self
  4. API-post-database-query
  5. API-post-search

🎉 Ready to create AutoGen agent with 19 Notion tools!
✅ Connected successfully!
📋 Available tools: 19

🛠️  Sample tools:
  1. API-get-user
  2. API-get-users
  3. API-get-self
  4. API-post-database-query
  5. API-post-search

🎉 Ready to create AutoGen agent with 19 Notion tools!


In [19]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv, find_dotenv
import os

# Load environment variables
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("❌ OPENAI_API_KEY not found in environment")
else:
    print("✅ OpenAI API key loaded")

# Create AutoGen agent with Notion MCP tools
notion_agent = AssistantAgent(
    name="notion_assistant",  # Fixed: use underscore instead of hyphen
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini", 
        api_key=OPENAI_API_KEY
    ),
    system_message="""You are a helpful assistant that can interact with Notion using MCP tools. 

You can:
- Search for pages and databases
- Query databases 
- Retrieve user information
- Get page/block details
- Create and update content

Always be helpful and explain what you're doing when using tools.
""",
    tools=notion_tools
)

print(f"🤖 Created AutoGen agent '{notion_agent.name}' with {len(notion_tools)} Notion tools")
print("📝 Ready to interact with Notion!")

✅ OpenAI API key loaded
🤖 Created AutoGen agent 'notion_assistant' with 19 Notion tools
📝 Ready to interact with Notion!
